In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class ResidualLayer(nn.Module):
    def __init__(self, filters, kernal_size=3):
        super().__init__()

        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Conv2d(filters, filters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(filters),
        )

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        x = self.conv2d_sequential(x)
        x += residual
        x = self.relu(x)

        return x
    
class ConvolutionLayer(nn.Module):
    def __init__(self, infilters, outfilters, kernal_size=3):
        super().__init__()
        
        self.conv2d_sequential = nn.Sequential(                
            nn.Conv2d(infilters, outfilters, kernal_size, padding=(kernal_size - 1) // 2),
            nn.BatchNorm2d(outfilters),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.conv2d_sequential(x)
        return x
    
class PolicyHead(nn.Module):
    def __init__(self, filters):
        super().__init__()
        self.filters = filters

        self.head = nn.Sequential(
            nn.Conv2d(self.filters, 1, 1),
            nn.Flatten(),
            nn.BatchNorm1d(225),
            nn.ReLU(),
            nn.Linear(225, 225)
        )

    def forward(self, x):
        x = self.head(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, filters, feature_dimensions, residual_layers=5, kernal_size=3):
        super().__init__()

        self.conv_layer = ConvolutionLayer(feature_dimensions, filters, kernal_size=kernal_size)
        self.residual_layers = nn.ModuleList([ResidualLayer(filters, kernal_size=kernal_size) for _ in range(residual_layers)])
        self.policy_head = PolicyHead(filters)

    def forward(self, x):      
        x = self.conv_layer(x)
        for layer in self.residual_layers:
            x = layer(x)
        x = self.policy_head(x)
        
        return x
    
Filters = 128
Layers = 20
HistoryDepth = 8
KernalSize = 3

model = NeuralNetwork(Filters, HistoryDepth + 1, Layers, kernal_size=KernalSize)
#model = torch.compile(model)

In [ ]:
model.load_state_dict(torch.load('../Models/HumanModels/128f20l5kAllBut715e/14.pt'))
model.eval()

In [ ]:
example = np.zeros((1, 15, 15, 15), dtype=np.float32)
example = torch.tensor(example)

traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("traced_resnet_model.pt")